<a href="https://colab.research.google.com/github/vinayak2019/organic_chem/blob/main/Day1/Molecular_Orbitals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# @title Run this cell to install the necessary software. { display-mode: "form" }

#@markdown This should take a minute to run.

! pip install pyscf
! pip install rdkit
! pip install geometric
! pip install py3Dmol

import os
import sys
import pyscf
import py3Dmol
import matplotlib.pyplot as plt

from pyscf.hessian import thermo
from pyscf import gto, scf, dft, tools, lo
from pyscf.geomopt.geometric_solver import optimize

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
from IPython.core.display import Image
#@title Load molecule { run: "auto" }
#@markdown Select a molecule name you want to visualize the molecular orbitals.

d = {
    "ethene":"C=C",
    "but-1,3-diene":"C=CC=C",
    "benzene": "c1ccccc1"
}

name = "but-1,3-diene"  # @param ["ethene", "but-1,3-diene"]

smiles = d[name]
molecule_name = "molecule"
# drawing_style = "stick" #@param ['stick', 'sphere', 'line','carton']

# Establish RDKit mol object
rd_mol = Chem.MolFromSmiles(smiles)
rdmol_hs = Chem.rdmolops.AddHs(rd_mol)
im = Draw.MolsToGridImage([rdmol_hs])
AllChem.EmbedMolecule(rdmol_hs)
AllChem.MMFFOptimizeMolecule(rdmol_hs)
Chem.MolToXYZFile(rdmol_hs, "{}.xyz".format(molecule_name))
im



In [ ]:
from IPython.core.display import Image
#@title View 3D molecules molecule { run: "auto" }


Chem.MolToXYZFile(rdmol_hs, "{}.xyz".format(molecule_name))
with open("{}.xyz".format(molecule_name)) as f:
  xyz = f.read()
v = py3Dmol.view()
v.addModel(xyz, 'xyz')
v.setStyle({'stick':{}})
v.show()

In [ ]:
#@title Run DFT calculations

#@markdown The theory behind DFT is beyond this course. In brief, we are solving Schrodinger equation to get the energy. This process will need information about the hamiltonian operator and the wavefuntion of electron. The functional represents the form of Hamiltonian operator and the basis-set is the equation of wavefunction. The calculation will generate molecular orbitals and the wavefunctions are the atomic orbitals needed to generate the molecular orbitals.


functional = "B3LYP" #@param ["B3LYP", "wb97x"]
basis_set = "3-21G" #@param ["3-21G","6-31G"]

# Set parameters
mol = gto.M(atom=F'{molecule_name}.xyz',  # Establish PySCF mol object
            basis = basis_set,
            verbose=0
            )
mf = mol.KS()
mf.xc = functional  # Set functional

# Run Optimization
# mol_eq = optimize(mf)
# # Save results to file
# mol_eq.tofile(F'{molecule_name}_optimization.xyz')

mf.kernel()

n = mol.tot_electrons()
homo_idx = int(n/2) -1
lumo_idx = int(n/2)
_ = tools.cubegen.orbital(mol, f'{molecule_name}_homo-1.cube', mf.mo_coeff[:,homo_idx-1],  nx=60, ny=60, nz=60)
_ = tools.cubegen.orbital(mol, f'{molecule_name}_homo.cube', mf.mo_coeff[:,homo_idx],  nx=60, ny=60, nz=60)
_ = tools.cubegen.orbital(mol, f'{molecule_name}_lumo.cube', mf.mo_coeff[:,lumo_idx],  nx=60, ny=60, nz=60)
_ = tools.cubegen.orbital(mol, f'{molecule_name}_lumo+1.cube', mf.mo_coeff[:,lumo_idx+1],  nx=60, ny=60, nz=60)


print("Calculations complete")
print("The energy of the molecule is {} Hartree".format(round(mf.e_tot,3)))

In [ ]:
#@title View Orbitals {run: "auto"}
#@markdown Select which molecular orbital to view


orbital = "lumo" #@param ["homo-1", "homo", "lumo", "lumo+1"] {type:"string"}


def draw_orbital(molecule_name, xyz, orb):
    view = py3Dmol.view(width=400,height=400)
    with open(f"./{molecule_name}_{orb}.cube") as f:
        cube_data = f.read()
    view.addVolumetricData(cube_data, "cube", {'isoval': -0.04, 'color': "red", 'opacity': 0.75})
    view.addVolumetricData(cube_data, "cube", {'isoval': 0.04, 'color': "blue", 'opacity': 0.75})
    view.addModel(xyz, 'xyz')
    view.setStyle({'stick':{}})
    view.zoomTo()
    view.update()
    view.clear()
    view.show()

draw_orbital(molecule_name, xyz, orbital)

The blue and red indicate the regions where you will likely find the electron. The Highest Occupied Molecular Orbital (HOMO) is the last molecular orbital of highest that contains electrons. LUMO is Lowest Unoccupied Molecular Orbital